## SCDGL_new  and SCGDL_Train_new Testing

 ## SPGDL_utils.py 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import sklearn.neighbors
from sklearn.mixture import BayesianGaussianMixture
from sklearn.metrics.cluster import adjusted_rand_score
import scipy.sparse as sp
import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torch_geometric.data import Data

def Adata2Torch_data(adata): 
    G_df = adata.uns['Spatial_Net'].copy() 
    spots = np.array(adata.obs_names) 
    spots_id_tran = dict(zip(spots, range(spots.shape[0]))) 
    G_df['Spot1'] = G_df['Spot1'].map(spots_id_tran) 
    G_df['Spot2'] = G_df['Spot2'].map(spots_id_tran) 

    G = sp.coo_matrix((np.ones(G_df.shape[0]), (G_df['Spot1'], G_df['Spot2'])), shape=(adata.n_obs, adata.n_obs))
    G = G + sp.eye(G.shape[0])

    edgeList = np.nonzero(G) 
    if type(adata.X) == np.ndarray:
        data = Data(edge_index=torch.LongTensor(np.array(
            [edgeList[0], edgeList[1]])), x=torch.FloatTensor(adata.X))  
    else:
        data = Data(edge_index=torch.LongTensor(np.array(
            [edgeList[0], edgeList[1]])), x=torch.FloatTensor(adata.X.todense()))  
    return data

def Spatial_Dis_Cal(adata, rad_dis=None, knn_dis=None, model='Radius', verbose=True):
    """\
    Calculate the spatial neighbor networks, as the distance between two spots.
    Parameters
    ----------
    adata:  AnnData object of scanpy package.
    rad_dis:  radius distance when model='Radius' 
    knn_dis:  The number of nearest neighbors when model='KNN'
    model:
        The network construction model. When model=='Radius', the spot is connected to spots whose distance is less than rad_dis. 
        When model=='KNN', the spot is connected to its first knn_dis nearest neighbors.
    Returns
    -------
    The spatial networks are saved in adata.uns['Spatial_Net']
    """
    assert(model in ['Radius', 'KNN']) 
    if verbose:
        print('------Calculating spatial graph...')
    coor = pd.DataFrame(adata.obsm['spatial']) 
    coor.index = adata.obs.index 
    coor.columns = ['Spatial_X', 'Spatial_Y'] 

    if model == 'Radius':
        nbrs = sklearn.neighbors.NearestNeighbors(radius=rad_dis).fit(coor)
        distances, indices = nbrs.radius_neighbors(coor, return_distance=True)
        KNN_list = []
        for spot in range(indices.shape[0]):
            KNN_list.append(pd.DataFrame(zip([spot]*indices[spot].shape[0], indices[spot], distances[spot]))) 
    
    if model == 'KNN':
        nbrs = sklearn.neighbors.NearestNeighbors(n_neighbors=knn_dis+1).fit(coor)
        distances, indices = nbrs.kneighbors(coor)
        KNN_list = []
        for spot in range(indices.shape[0]):
            KNN_list.append(pd.DataFrame(zip([spot]*indices.shape[1],indices[spot,:], distances[spot,:])))

    KNN_df = pd.concat(KNN_list) 
    KNN_df.columns = ['Spot1', 'Spot2', 'Distance']

    Spatial_Net = KNN_df.copy()
    Spatial_Net = Spatial_Net.loc[Spatial_Net['Distance']>0,]
    id_spot_trans = dict(zip(range(coor.shape[0]), np.array(coor.index), )) 
    Spatial_Net['Spot1'] = Spatial_Net['Spot1'].map(id_spot_trans) 
    Spatial_Net['Spot2'] = Spatial_Net['Spot2'].map(id_spot_trans) 
    if verbose:
        print('The graph contains %d edges, %d spots.' %(Spatial_Net.shape[0], adata.n_obs)) 
        print('%.4f neighbors per spot on average.' %(Spatial_Net.shape[0]/adata.n_obs)) 

    adata.uns['Spatial_Net'] = Spatial_Net

def Spatial_Dis_Draw(adata):
    import matplotlib.pyplot as plt
    Num_edge = adata.uns['Spatial_Net']['Spot1'].shape[0] 
    Mean_edge = Num_edge/adata.shape[0] 
    plot_df = pd.value_counts(pd.value_counts(adata.uns['Spatial_Net']['Spot1'])) 
    plot_df = plot_df/adata.shape[0]  
    fig, ax = plt.subplots(figsize=[4,4],dpi=300)
    plt.ylabel('Percentage')
    plt.xlabel('Edge Numbers per Spot')
    plt.title('Number of Neighbors for Spots (Average=%.2f)'%Mean_edge)
    ax.bar(plot_df.index, plot_df,color="#aa40fc",edgecolor="#f7b6d2",linewidth=2)

def Cal_Spatial_variable_genes(adata):
    import SpatialDE
    counts = pd.DataFrame(adata.X, columns=adata.var_names, index=adata.obs_names)
    coor = pd.DataFrame(adata.obsm['spatial'], columns=['Spatial_X', 'Spatial_Y'], index=adata.obs_names)
    Spatial_var_genes = SpatialDE.run(coor, counts)
    Spatial_3000_var_genes = Spatial_var_genes["g"].values[0:3000]
    Spatial_3000_var_genes = pd.DataFrame(Spatial_3000_var_genes)
    all_genes = counts.columns.to_frame()
    for i in range(len(all_genes.values)):
        if all_genes.values[i] in Spatial_3000_var_genes.values:
            all_genes.values[i] =1
        else:
            all_genes.values[i] =0
    Spatial_highly_genes = all_genes.squeeze()
    adata.var["Spatial_highly_variable_genes"] = Spatial_highly_genes.astype(bool)

def DGI_loss_Draw(adata):
    import matplotlib.pyplot as plt
    if "SCGDL_loss" not in adata.uns.keys():
        raise ValueError("Please Train DGI Model using SCGDL_Train function first!") 
    Train_loss = adata.uns["SCGDL_loss"]
    plt.style.use('default') 
    plt.plot(Train_loss,label='Training loss',linewidth=2)
    plt.xlabel("Number of Epochs")
    plt.ylabel("Loss of DGI model")
    plt.legend()
    plt.grid()

def BGMM(adata,n_cluster,used_obsm='SCGDL'):
    """
    BayesianGaussianMixture for spatial clustering.
    """

    knowledge = BayesianGaussianMixture(n_components=n_cluster,
                                        weight_concentration_prior_type ='dirichlet_process', ##'dirichlet_process' or dirichlet_distribution'
                                        weight_concentration_prior = 50).fit(adata.obsm[used_obsm])                                  
    # load ground truth for ARI and NMI computation.
    Ann_df = pd.read_csv("/home/tengliu/Torch_pyG/SCGDL_Upload_Files/data/Human_DLPFC/151676_truth.txt", sep='\t', header=None, index_col=0)
    Ann_df.columns = ['Ground Truth']
    adata.obs['Ground Truth'] = Ann_df.loc[adata.obs_names, 'Ground Truth']

    method = "BayesianGaussianMixture"
    labels = knowledge.predict(adata.obsm[used_obsm])+1
    Ann_df.columns = [method] 
    Ann_df.loc[:,method] = labels 
    adata.obs[method] = Ann_df.loc[adata.obs_names, method] 
    adata.obs[method] = adata.obs[method].astype('category') 
    obs_df = adata.obs.dropna() 
    ARI = adjusted_rand_score(obs_df[method], obs_df['Ground Truth']) 
    adata.uns["ARI"] = ARI 
    return adata


In [ ]:
import scanpy as sc
adata=sc.read('/home/tengliu/Torch_pyG/SCGDL_Upload_Files/data/Human_DLPFC/151676_Anndata.h5ad')

In [ ]:
adata

In [ ]:
plt.rcParams["figure.figsize"] = (4, 4)
sc.pl.spatial(adata, img_key="hires", color=["Ground Truth"])

In [ ]:
Spatial_Dis_Cal(adata,rad_dis=150)
Spatial_Dis_Draw(adata)

## SCGDL_Train_new.py

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm 
import scipy.sparse as sp

from SCGDL import ResGatedGraphmodel
from SCGDL_auxiliary import Adata2Torch_data

import torch
import torch.backends.cudnn as cudnn
cudnn.deterministic = True 
cudnn.benchmark = True 
import torch.nn.functional as F 
from torch_geometric.nn import DeepGraphInfomax

class my_data():
    def __init__(self, x, edge_index, edge_attr):
        self.x = x
        self.edge_index = edge_index
        self.edge_attr = edge_attr

def corruption(data): 
    x = data.x[torch.randperm(data.x.size(0))] 
    return my_data(x, data.edge_index, data.edge_attr)


def SCGDL_Train(adata, 
                hidden_dims=[128, 128], 
                num_epochs=1000, 
                lr=1e-6, 
                key_added='SCGDL',
                gradient_clipping=5., 
                weight_decay=0.0001, 
                random_seed=0, save_loss=True):
    """\
    Training graph attention auto-encoder.
    Parameters
    ----------
    adata: AnnData object of scanpy package.
    hidden_dims: The dimension of the encoder.
    n_epochs:Number of total epochs for training.
    lr: Learning rate for AdamOptimizer.
    key_added: The latent embeddings are saved in adata.obsm[key_added].
    gradient_clipping: Gradient Clipping. 
    weight_decay: Weight decay for AdamOptimizer.
    save_loss: If True, the training loss is saved in adata.uns['SCGDL_loss'].
    save_reconst_exp: If True, the reconstructed expression profiles are saved in adata.layers['SCGDL_ReX'].
    device: See torch.device.

    Returns
    -------
    AnnData
    """
    seed=random_seed
    import random
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    adata.X = sp.csr_matrix(adata.X)

    if "Spatial_highly_variable_genes" in adata.var.columns:
        adata_Vars =  adata[:, adata.var['Spatial_highly_variable_genes']]
        print('Input Size using Spatial_variable_genes: ', adata_Vars.shape)
    elif 'highly_variable' in adata.var.columns:
        adata_Vars =  adata[:, adata.var['highly_variable']]
        print('Input Size using Highly_variable_genes: ', adata_Vars.shape)
    else:
        adata_Vars = adata
        print('Input Size using All genes list: ', adata_Vars.shape) 

    if 'Spatial_Net' not in adata.uns.keys():
        raise ValueError("Please Compute Spatial Network using Spatial_Dis_Cal function first!") 

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    data = Adata2Torch_data(adata_Vars) 
    hidden_dims = [data.x.shape[1]] + hidden_dims 
  
    DGI_model = DeepGraphInfomax(
        hidden_channels=hidden_dims[1], 
        encoder=ResGatedGraphmodel(hidden_dims), 
        summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)), 
        corruption=corruption).to(device) 
    DGI_optimizer = torch.optim.Adam(DGI_model.parameters(), lr=lr, weight_decay=weight_decay)
    data = data.to(device)

    import time
    start_time = time.time()
    loss_list = []
    for epoch in tqdm(range(1, num_epochs+1)):
        DGI_model.train()
        DGI_optimizer.zero_grad() 
        pos_z, neg_z, summary = DGI_model(data=data) 
        DGI_loss = DGI_model.loss(pos_z, neg_z, summary) 
        loss_list.append(DGI_loss.item())
        DGI_loss.backward()
        torch.nn.utils.clip_grad_norm_(DGI_model.parameters(), gradient_clipping) 
        DGI_optimizer.step()
        if ((epoch)%1000) == 0:
            print('Epoch: {:03d}, Loss: {:.4f}'.format(epoch, np.mean(loss_list)))
    end_time = time.time()
    print('Elapsed training time:{:.4f} seconds'.format((end_time-start_time)))

    DGI_model.eval()
    pos_z, neg_z, summary = DGI_model(data=data) 

    SCGDL_rep = pos_z.to('cpu').detach().numpy() 
    adata.obsm[key_added] = SCGDL_rep

    if save_loss:
        adata.uns['SCGDL_loss'] = loss_list

    return adata

In [ ]:

adata = SCGDL_Train(adata,hidden_dims=[128, 128],num_epochs=1000)

In [ ]:
DGI_loss_Draw(adata) #16 dim: 1.1579; 32 dim:0.5659; 64 dim:0.6782; 128 dim: 0.7261; 256 dim：1.9262; 512 dim: 2.2487

In [ ]:
adata.obsm["SCGDL"].shape

In [ ]:
sc.pp.neighbors(adata, use_rep='SCGDL')
# sc.pp.neighbors(adata, use_rep='STAGATE',n_neighbors=250,n_pcs=30,knn=True,method="umap") 
sc.tl.umap(adata) 
sc.pl.umap(adata,color="Ground Truth") 

In [ ]:
adata = BGMM(adata,n_cluster=7,used_obsm='SCGDL')

In [ ]:
plt.rcParams["figure.figsize"] = (4, 4)
sc.pl.umap(adata, color=["BayesianGaussianMixture", "Ground Truth"],legend_loc='on data', 
            legend_fontsize=12,legend_fontoutline=2,frameon=False, title=["BayesianGaussianMixture" + ' (ARI=%.2f)'%adata.uns["ARI"], "Ground Truth"])